# Toy Data: 8-Modes Mixture Model

## Introduction

This notebook is supposed to demonstrate how to use FrEIA to create reversible architectures.

We will use the toy data from the paper in this example.

In [1]:
%matplotlib notebook

import torch
import torch.optim

from scipy import stats

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from tqdm import tqdm
from time import time

import scipy.io as sio

import FrEIA

from FrEIA.framework import InputNode, OutputNode, Node, ReversibleGraphNet
from FrEIA.modules import rev_gru_layer, F_GRU, glow_gru_coupling_layer, F_fully_connected, glow_gru_cgate_coupling_layer2, F_cgate

import data

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)


# Easy case: 30:30:30 vs. 30:26:22. rng(0) iter: 3000
# torch.manual_seed(1)
# N_samp = 100
# ndim_tot = 104
# ndim_z = 10

# Hard case: 30:30:30 vs. 30:28:26. rng(2) iter: 3000
torch.manual_seed(0)
N_samp = 100
ndim_tot = 150
ndim_z = 4

cuda


In [2]:
X_mat = sio.loadmat('data/dx_av45/CDRSB_AV45_AV45.mat')
pos = X_mat['X']
print(pos.shape)
print(type(pos))

Y_mat = sio.loadmat('data/dx_av45/CDRSB_AV45_CDRSB.mat')
labels_onehot = Y_mat['Y']
print(labels_onehot.shape)
print(type(labels_onehot))

Y_name_mat = sio.loadmat('data/dx_av45/AV45_ROI_names.mat')
Y_name = np.squeeze(Y_name_mat['av45_names'][0])
print(Y_name[0][0])
print(Y_name[1][0])

pos = torch.tensor(pos).float()
labels_onehot = np.expand_dims(labels_onehot, 2)
labels_onehot = torch.tensor(labels_onehot).float()

print(labels_onehot.shape)

(279, 3, 82)
<class 'numpy.ndarray'>
(279, 3)
<class 'numpy.ndarray'>
CTX_LH_CAUDALANTERIORCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_CAUDALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16
torch.Size([279, 3, 1])


In [3]:
batch_size = labels_onehot.data.numpy().shape[0]
T = labels_onehot.data.numpy().shape[1]
ndim_x = pos.data.numpy().shape[2]
ndim_y = labels_onehot.data.numpy().shape[2]

print("ndim_x: ", ndim_x)
print("ndim_y: ", ndim_y)
print("ndim_z: ", ndim_z)

P_Z = multivariate_normal(mean=np.zeros(ndim_z))

inp = InputNode(ndim_tot, name='input')

# t1 = Node([inp.out0], rev_gru_layer,
#           {'F_class': F_GRU_cgate, 'clamp': 2.0,
#            'F_args': {'dropout': 0.0}})

# t2 = Node([t1.out0], rev_gru_layer,
#           {'F_class': F_GRU_cgate, 'clamp': 2.0,
#            'F_args': {'dropout': 0.0}})

# t3 = Node([t2.out0], rev_gru_layer,
#           {'F_class': F_GRU_cgate, 'clamp': 2.0,
#            'F_args': {'dropout': 0.0}})

# t1 = Node([inp.out0], glow_gru_cgate_coupling_layer,
#           {'F_class': F_GRU, 'clamp': 2.0,
#            'F_args': {'dropout': 0.0}})

# t2 = Node([t1.out0], glow_gru_cgate_coupling_layer,
#           {'F_class': F_GRU, 'clamp': 2.0,
#            'F_args': {'dropout': 0.0}})

t1 = Node([inp.out0], glow_gru_cgate_coupling_layer2,
          {'F_class': F_GRU, 'F_cgate': F_cgate, 'clamp': 2.0,
           'F_args': {'dropout': 0.0}})

t2 = Node([t1.out0], glow_gru_cgate_coupling_layer2,
          {'F_class': F_GRU, 'F_cgate': F_cgate, 'clamp': 2.0,
           'F_args': {'dropout': 0.0}})

outp = OutputNode([t2.out0], name='output')

nodes = [inp, t1, t2, outp]
model = ReversibleGraphNet(nodes)
model = model.cuda()

ndim_x:  82
ndim_y:  1
ndim_z:  4
Node 2f6dd8 has following input dimensions:
	 Output #0 of node input: (150,)

Node 2f6eb8 has following input dimensions:
	 Output #0 of node 2f6dd8: (150,)

Node output has following input dimensions:
	 Output #0 of node 2f6eb8: (150,)



In [4]:
# Training parameters
n_epochs = 3000
meta_epoch = 12
n_its_per_epoch = 10

lr = 1e-4
gamma = 0.01**(1./120)
l2_reg = 2e-5

y_noise_scale = 3e-3
zeros_noise_scale = 3e-2
# y_noise_scale = 0
# zeros_noise_scale = 0

# relative weighting of losses:
# lambd_predict = 300.
# lambd_latent = 300.
# lambd_rev = 400.
lambd_predict = 300.
lambd_latent = 300.
lambd_rev = 100.

pad_x = torch.zeros(batch_size, ndim_tot - ndim_x)
pad_yz = torch.zeros(batch_size, ndim_tot - ndim_y - ndim_z)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999),
                             eps=1e-08, weight_decay=l2_reg)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=meta_epoch, gamma=gamma)


def MMD_multiscale(x, y):
    xx, yy, zz = torch.mm(x,x.t()), torch.mm(y,y.t()), torch.mm(x,y.t())

    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))

    dxx = rx.t() + rx - 2.*xx
    dyy = ry.t() + ry - 2.*yy
    dxy = rx.t() + ry - 2.*zz

    XX, YY, XY = (torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device))

    for a in [0.2, 0.5, 0.9, 1.3]:
        XX += a**2 * (a**2 + dxx)**-1
        YY += a**2 * (a**2 + dyy)**-1
        XY += a**2 * (a**2 + dxy)**-1

    return torch.mean(XX + YY - 2.*XY)


def fit(input, target):
    return torch.mean((input - target)**2)

loss_backward = MMD_multiscale
loss_latent = MMD_multiscale
loss_fit = fit

train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(pos, labels_onehot),
    batch_size=batch_size, shuffle=True, drop_last=True)

print(train_loader.dataset.tensors[0].shape)
print(train_loader.dataset.tensors[1].shape)

x_full = train_loader.dataset.tensors[0].float()
y_full = train_loader.dataset.tensors[1].float()
# y_full = y_full / 30

torch.Size([279, 3, 82])
torch.Size([279, 3, 1])


In [5]:
def train(i_epoch=0):
    model.train()

    l_tot = 0
    batch_idx = 0
    
    t_start = time()
    
    loss_factor = 600**(float(i_epoch) / 300) / 600
    if loss_factor > 1:
        loss_factor = 1
    
    loss = 0
    for x, y in train_loader:
        batch_idx += 1
        if batch_idx > n_its_per_epoch:
            break

        x, y = x.to(device), y.to(device)
        y_clean = y.clone()
        pad_x = zeros_noise_scale * torch.randn(batch_size, T, ndim_tot - ndim_x, device=device)
        pad_yz = zeros_noise_scale * torch.randn(batch_size, T, ndim_tot - ndim_y - ndim_z, device=device)

        y += y_noise_scale * torch.randn(batch_size, T, ndim_y, dtype=torch.float, device=device)

        x, y = (torch.cat((x, pad_x),  dim=2),
                torch.cat((torch.randn(batch_size, T, ndim_z, device=device), pad_yz, y), dim=2))


        optimizer.zero_grad()
        
        # Forward step:

#         print("x.shape: ", x.shape)
#         print("y.shape: ", y.shape)
        
        output = model(x)
#         print(x.shape, x.iscuda, y.shape, y.iscuda)
#         print('-------------------------------')
        l = 0
        for t in range(T):

            # L_y( y, y_gt)
            l += lambd_predict * loss_fit(output[:, t, ndim_z:], y[:, t, ndim_z:])

            # Shorten output, and remove gradients wrt y, for latent loss
            y_short = torch.cat((y[:, t, :ndim_z], y[:, t, -ndim_y:]), dim=1)

            output_block_grad = torch.cat((output[:, t, :ndim_z],
                                           output[:, t, -ndim_y:].data), dim=1)

            # L_z( [y,z], [y_gt, z_sample] )
            l += lambd_latent * loss_latent(output_block_grad, y_short)
            
        l_tot += l.data.item()

        l.backward()

        # Backward step:
        pad_yz = zeros_noise_scale * torch.randn(batch_size, T, ndim_tot - ndim_y - ndim_z, device=device)
        y = y_clean + y_noise_scale * torch.randn(batch_size, T, ndim_y, device=device)

        orig_z_perturbed = (output.data[:, :, :ndim_z] + y_noise_scale *
                            torch.randn(batch_size, T, ndim_z, device=device))
        
        # perturbed [z=f_z(x), pad, y_gt]
        y_rev = torch.cat((orig_z_perturbed, pad_yz, y), dim=2)
        # perturbed [z_sample, pad, y_gt]
        y_rev_rand = torch.cat((torch.randn(batch_size, T, ndim_z, device=device), pad_yz, y), dim=2)
        
        output_rev = model(y_rev, rev=True)
        output_rev_rand = model(y_rev_rand, rev=True)

        # L_x (MMD)
        l_rev = 0
        for t in range(T):
            l_rev += lambd_rev * loss_factor * loss_backward(output_rev_rand[:, t, :ndim_x], x[:, t, :ndim_x])

            # MSE fit loss against X -> [Y,Z]+perturb -> X_inv and original X
            # MSE( g([output_z_pert, pad, y_gt_pert]), [x, pad] )
            l_rev += 0.50 * lambd_predict * loss_fit(output_rev[:,t,:], x[:,t,:])
    #         l_rev += 0.50 * lambd_predict * loss_backward(output_rev, x)

        l_tot += l_rev.data.item()
        l_rev.backward()

        for p in model.parameters():
            p.grad.data.clamp_(-5.00, 5.00)
#             p.grad.data.clamp_(-15.00, 15.00)

        optimizer.step()

#     print('%.1f\t%.5f' % (
#                              float(batch_idx)/(time()-t_start),
#                              l_tot / batch_idx,
#                            ), flush=True)
        loss += l_tot
#     print('loss:\t%.5f' % (loss), flush=True)

    return l_tot / batch_idx

In [6]:
def log_e(s):
    '''log of the nonlinear function e'''
    return 2.0 * 0.636 * np.arctan(s)

def e(s):
    return np.exp(2.0 * 0.636 * np.arctan(s))

In [7]:


y_temp = np.zeros((N_samp, T, ndim_y))
y1 = y_temp
y1[:,0,0] = 0
y1[:,1,0] = 0
y1[:,2,0] = 0
# y1 = y1 / 30
print(y1[:2])

y_temp = np.zeros((N_samp, T, ndim_y))
y2 = y_temp
# y2[:,0,0] = 30
# y2[:,1,0] = 28
# y2[:,2,0] = 26
y2[:,0,0] = 0
y2[:,1,0] = 5
y2[:,2,0] = 10
# y2 = y2 / 30
print(y2[:2])

y1 = torch.tensor(y1).float()
y2 = torch.tensor(y2).float()

[[[0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]]]
[[[ 0.]
  [ 5.]
  [10.]]

 [[ 0.]
  [ 5.]
  [10.]]]


In [8]:
 # for mod_list in model.children():
#     for block in mod_list.children():
#         for coeff in block.children():
#             coeff.fc3.weight.data = 0.01*torch.randn(coeff.fc3.weight.shape)

model = model.cuda()
model.to(device)

# First set up the test samples that we use consistently to see the change over training.
y1 += y_noise_scale * torch.randn(N_samp, T, ndim_y)
y2 += y_noise_scale * torch.randn(N_samp, T, ndim_y)
z_samps = torch.randn(N_samp, T, ndim_z)
yz_pads = zeros_noise_scale * torch.zeros(N_samp, T, ndim_tot - ndim_y - ndim_z)
z1_samps = torch.randn(N_samp, T, ndim_z)
z2_samps = torch.randn(N_samp, T, ndim_z)
y1_samps = torch.cat([z1_samps, yz_pads,  y1], dim=2)
y2_samps = torch.cat([z2_samps, yz_pads,  y2], dim=2)
y1_samps = y1_samps.to(device)
y2_samps = y2_samps.to(device)

try:
    t_start = time()
    for i_epoch in tqdm(range(n_epochs), ascii=True, ncols=80):

        scheduler.step()

        # Initially, the l2 reg. on x and z can give huge gradients, set
        # the lr lower for this
        if i_epoch < 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr * 1e-2

        train(i_epoch)
        
        if i_epoch%500 == 0:

            rev_x1 = model(y1_samps, rev=True)
            rev_x2 = model(y2_samps, rev=True)

            x1_gen = rev_x1[:,:,:ndim_x].data.cpu().numpy()
            x2_gen = rev_x2[:,:,:ndim_x].data.cpu().numpy()

            t_test = stats.ttest_ind(x1_gen, x2_gen, axis=0)

            print("means")
            print(np.mean(x1_gen[:,2,:5], axis=0))
            print(np.mean(x2_gen[:,2,:5], axis=0))

            # print(t_test.pvalue.shape)
            # print(t_test.pvalue)
            alpha = 0.01 / ndim_x

            for tt in range(T):
                pvals = t_test.pvalue[tt]
                sig_idx = pvals < alpha
                num_sig_rois = np.sum(sig_idx)
                print("t_test ROI")
                print("t:", tt+1, "found ", num_sig_rois, "sig rois")
                if num_sig_rois > 0:
                    for jj in range(ndim_x):
                        if sig_idx[jj] == True:
                            aa = np.mean(x_full[:,tt,jj].data.cpu().numpy(), axis=0)
                            bb = np.mean(x1_gen[:,tt,jj], axis=0)
                            cc = np.mean(x2_gen[:,tt,jj], axis=0)
                            print(Y_name[jj][0], aa, bb, cc)

            sio.savemat('/home/seong/Dropbox/research/iccv2019/desikan_rois/results_mat/cdr_results_i'+str(i_epoch)+'.mat', {'x1_gen':x1_gen,'x2_gen':x2_gen, 'pvals':pvals})

            x1_gen_delta10 = x1_gen[:,1,:] - x1_gen[:,0,:]   
            x1_gen_delta21 = x1_gen[:,2,:] - x1_gen[:,1,:]  
            x1_gen_delta20 = x1_gen[:,2,:] - x1_gen[:,0,:]     
            x1_gen_delta = np.stack((x1_gen_delta10, x1_gen_delta21), axis=1)

            x2_gen_delta10 = x2_gen[:,1,:] - x2_gen[:,0,:]   
            x2_gen_delta21 = x2_gen[:,2,:] - x2_gen[:,1,:]    
            x2_gen_delta20 = x2_gen[:,2,:] - x2_gen[:,0,:]     
            x2_gen_delta = np.stack((x2_gen_delta10, x2_gen_delta21), axis=1)
    #         t_test_delta = stats.ttest_ind(x1_gen_delta, x2_gen_delta, axis=0)


            t_test_delta = stats.ttest_ind(x1_gen_delta20, x2_gen_delta20, axis=0)

#             pvals = t_test_delta.pvalue
#             sig_idx = pvals < alpha
#             num_sig_rois = np.sum(sig_idx)
#             print("t_test_delta ROI")
#             print("t2 - t0: ", "found ", num_sig_rois, "sig rois")
#             if num_sig_rois > 0:
#                 for jj in range(ndim_x):
#                     if sig_idx[jj] == True:
#                         print(Y_name[jj][0])
                        
        
#         for tt in range(T-1):
#             pvals = t_test_delta.pvalue[tt]
#             sig_idx = pvals < alpha
#             num_sig_rois = np.sum(sig_idx)
#             print("t_test_delta ROI")
#             print("t:", tt+1, "found ", num_sig_rois, "sig rois")
#             if num_sig_rois > 0:
#                 for jj in range(ndim_x):
#                     if sig_idx[jj] == True:
#                         print(Y_name[jj][0])
        
#         t_test_delta = stats.ttest_ind(x1_gen_delta, x2_gen_delta, axis=0)
        
        # print(t_test.pvalue.shape)
#         # print(t_test.pvalue)
#         for tt in range(T-1):
#             pvals = t_test_delta.pvalue[tt]
#             sig_idx = pvals < alpha
#             num_sig_rois = np.sum(sig_idx)
#             print("t_test_delta ROI")
#             print("t:", tt+1, "found ", num_sig_rois, "sig rois")
#             if num_sig_rois > 0:
#                 for jj in range(ndim_x):
#                     if sig_idx[jj] == True:
#                         print(Y_name[jj][0])
        
        


except KeyboardInterrupt:
    pass
finally:
    print(f"\n\nTraining took {(time()-t_start)/60:.2f} minutes\n")

  0%|                                          | 1/3000 [00:00<13:50,  3.61it/s]

means
[ 0.15473439 -0.61951464 -0.05037158 -0.12093434  0.12524389]
[-0.09351673  0.0803297  -0.02643159  0.08565709  0.178185  ]
t_test ROI
t: 1 found  0 sig rois
t_test ROI
t: 2 found  71 sig rois
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16 1.0934255 0.18741177 0.18880597
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16 1.3445065 0.056327246 0.08211392
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16 1.3725337 0.1260546 0.12863521
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16 1.3253946 -0.0479801 -0.08705439
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16 1.3439506 0.18100052 0.22116919
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16 1.4760946 0.22923347 0.3181684
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16 1.2976238 -0.09721314 -0.08167338
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.4289154 -0.025750875 -0.011302853
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16 1.2929226 -0.05731497 -0.06248462
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.3319682 0.09918685 0.113646634
CTX_LH_MIDDLETEMP

 17%|######6                                 | 501/3000 [01:18<07:37,  5.47it/s]

means
[1.3872715 1.2429214 1.2436688 1.006584  0.9450281]
[1.5380582 1.5527802 1.3631239 1.0441903 1.6218555]
t_test ROI
t: 1 found  0 sig rois
t_test ROI
t: 2 found  65 sig rois
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16 1.0934255 0.96530366 1.3230706
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16 1.3445065 1.2421854 1.4066814
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16 1.3725337 1.2213578 1.4936751
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16 1.3253946 1.2130545 1.431747
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16 1.3439506 1.2561092 1.3676049
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16 1.4760946 1.3382245 1.5327976
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16 1.2976238 1.1755623 1.3974288
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.4289154 1.3207735 1.5260625
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16 1.2929226 1.2047557 1.336373
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.3319682 1.2052876 1.4877759
CTX_LH_MIDDLETEMPORAL_UCBERKELEYAV45_10_17_16 1.2734368 1.1409954 1

 33%|#############                          | 1001/3000 [02:26<07:30,  4.44it/s]

...

means
[1.3898633  1.2563434  1.2569678  1.0172184  0.94641274]
[1.6793915 1.6349068 1.4277182 1.0770752 1.5370466]
t_test ROI
t: 1 found  0 sig rois
t_test ROI
t: 2 found  65 sig rois
CTX_LH_CAUDALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16 1.4081545 1.2999104 1.4614545
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16 1.0934255 0.9604443 1.342196
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16 1.3445065 1.2604418 1.4701822
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16 1.3725337 1.2385651 1.5794683
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16 1.3253946 1.2174777 1.5019537
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16 1.3439506 1.278974 1.4146347
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16 1.4760946 1.3814104 1.5925584
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16 1.2976238 1.182416 1.4876072
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.4289154 1.3313322 1.5599906
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16 1.2929226 1.2235191 1.4017605
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.3319682 1.2

 83%|################################5      | 2501/3000 [06:16<01:42,  4.86it/s]

means
[1.3901138 1.2566628 1.2570379 1.0181983 0.9473316]
[1.6804639 1.6341368 1.4274672 1.0780436 1.5367639]
t_test ROI
t: 1 found  0 sig rois
t_test ROI
t: 2 found  65 sig rois
CTX_LH_CAUDALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16 1.4081545 1.300809 1.4624766
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16 1.0934255 0.96229 1.3443408
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16 1.3445065 1.2639371 1.4746289
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16 1.3725337 1.2418789 1.5840529
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16 1.3253946 1.2211316 1.5072777
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16 1.3439506 1.2825581 1.4187206
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16 1.4760946 1.3849099 1.5968268
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16 1.2976238 1.1833185 1.4896326
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.4289154 1.3348727 1.5637467
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16 1.2929226 1.2253717 1.4047627
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16 1.3319682 1.228723

100%|#######################################| 3000/3000 [07:34<00:00,  7.22it/s]



Training took 7.58 minutes



In [9]:
t_test = stats.ttest_ind(x1_gen, x2_gen, axis=0)
# print(t_test.pvalue.shape)
# print(t_test.pvalue)
alpha = 0.01 / ndim_x
for tt in range(T):
    pvals = t_test.pvalue[tt]
    sig_idx = pvals < alpha
    num_sig_rois = np.sum(sig_idx)
    print("t:", tt+1, "found ", num_sig_rois, "sig rois")
    if num_sig_rois > 0:
        for jj in range(ndim_x):
            if sig_idx[jj] == True:
                print(Y_name[jj][0])

sio.savemat('/home/seong/Dropbox/research/iccv2019/desikan_rois/results_mat/cdr_results_i3000.mat', {'x1_gen':x1_gen,'x2_gen':x2_gen, 'pvals':pvals})

print(pvals)

t: 1 found  0 sig rois
t: 2 found  65 sig rois
CTX_LH_CAUDALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_MIDDLETEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARACENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARSOPERCULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSORBITALIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSTRIANGULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PERICALCARINE_UCBERKELEYAV45_10_17_16
CTX_LH_POSTCENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_POSTERIORCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_PRECENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_PRECUNEUS_UCBERKELEYAV45_10_17_

In [10]:

x1_gen_delta10 = x1_gen[:,1,:] - x1_gen[:,0,:]   
x1_gen_delta21 = x1_gen[:,2,:] - x1_gen[:,1,:]    
x1_gen_delta = np.stack((x1_gen_delta10, x1_gen_delta21), axis=1)
x2_gen_delta10 = x2_gen[:,1,:] - x2_gen[:,0,:]   
x2_gen_delta21 = x2_gen[:,2,:] - x2_gen[:,1,:]    
x2_gen_delta = np.stack((x2_gen_delta10, x2_gen_delta21), axis=1)

t_test_delta = stats.ttest_ind(x1_gen_delta, x2_gen_delta, axis=0)

# print(t_test.pvalue.shape)
# print(t_test.pvalue)
alpha = 0.05 / ndim_x
for tt in range(T-1):
    pvals = t_test_delta.pvalue[tt]
    sig_idx = pvals < alpha
    num_sig_rois = np.sum(sig_idx)
    print("t_test_delta ROI")
    print("t:", tt+1, "found ", num_sig_rois, "sig rois")
    if num_sig_rois > 0:
        for jj in range(ndim_x):
            if sig_idx[jj] == True:
                print(Y_name[jj][0])
                
print(np.where(sig_idx==True))

print(np.mean(x1_gen_delta[:,1,np.where(sig_idx==True)], axis=0))
print(np.mean(x2_gen_delta[:,1,np.where(sig_idx==True)], axis=0))

t_test_delta ROI
t: 1 found  64 sig rois
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_MIDDLETEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARACENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARSOPERCULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSORBITALIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSTRIANGULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PERICALCARINE_UCBERKELEYAV45_10_17_16
CTX_LH_POSTCENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_POSTERIORCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_PRECENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_PRECUNEUS_UCBERKELEYAV45_10_17_16
CTX_LH_ROSTRALANTERIORCINGULATE_UCBERKELEYAV45_10_17_1

In [11]:
print(x1_gen[:10,2,:5])
print(x2_gen[:10,2,:5])
print(x_full[:10,2,:5])
print(np.mean(x1_gen[:,2,:5], axis=0))
print(np.mean(x2_gen[:,2,:5], axis=0))

[[1.6026887  1.1865268  1.2332534  0.9381481  0.8056886 ]
 [1.478961   1.7316537  1.5880839  1.2093109  1.2594937 ]
 [1.4469819  1.2688553  1.2239195  0.94677657 0.9373925 ]
 [1.4015741  1.4215381  1.4593557  0.968359   0.91230536]
 [1.8114831  1.6873771  1.3801286  1.078091   1.1298836 ]
 [1.0490719  0.8483167  1.0350938  0.7365536  0.78954786]
 [1.84874    1.7706467  1.4746457  1.2841336  1.5214448 ]
 [1.2121919  1.3477876  1.2650106  1.1066266  0.8830966 ]
 [1.337951   1.5106705  1.0303344  1.0913587  0.88518924]
 [1.569873   1.0124347  1.1135708  0.91868883 0.7843739 ]]
[[1.6408261  2.0288296  1.4923201  1.2124151  1.5472031 ]
 [1.7688483  1.5342826  1.4770185  1.0815635  1.5352402 ]
 [1.531154   1.2721984  1.4420784  1.2490488  1.47165   ]
 [1.7655452  1.2981317  1.4033575  1.1251788  1.5320722 ]
 [2.1016476  2.1412368  1.7259071  1.2721763  1.9186766 ]
 [1.2289363  1.416946   1.2176899  0.6493618  1.1985844 ]
 [0.9804624  1.4424973  1.0964266  1.0071576  1.3077186 ]
 [1.9706532  

In [12]:
x1_gen_delta21 = x1_gen[:,1,:] - x1_gen[:,0,:]
print(np.stack((x1_gen_delta21,x1_gen_delta21), axis=1).shape)


(100, 2, 82)


In [13]:
t_test_delta = stats.ttest_ind(x1_gen_delta20, x2_gen_delta20, axis=0)
        
pvals = t_test_delta.pvalue
sig_idx = pvals < alpha
num_sig_rois = np.sum(sig_idx)
print("t_test_delta ROI")
print("t2 - t0: ", "found ", num_sig_rois, "sig rois")
if num_sig_rois > 0:
    for jj in range(ndim_x):
        if sig_idx[jj] == True:
            print(Y_name[jj][0])
            
print(pvals)

t_test_delta ROI
t2 - t0:  found  76 sig rois
CTX_LH_CAUDALANTERIORCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_CAUDALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_CUNEUS_UCBERKELEYAV45_10_17_16
CTX_LH_FRONTALPOLE_UCBERKELEYAV45_10_17_16
CTX_LH_FUSIFORM_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORPARIETAL_UCBERKELEYAV45_10_17_16
CTX_LH_INFERIORTEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_INSULA_UCBERKELEYAV45_10_17_16
CTX_LH_ISTHMUSCINGULATE_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALOCCIPITAL_UCBERKELEYAV45_10_17_16
CTX_LH_LATERALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_LINGUAL_UCBERKELEYAV45_10_17_16
CTX_LH_MEDIALORBITOFRONTAL_UCBERKELEYAV45_10_17_16
CTX_LH_MIDDLETEMPORAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARACENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_PARSOPERCULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSORBITALIS_UCBERKELEYAV45_10_17_16
CTX_LH_PARSTRIANGULARIS_UCBERKELEYAV45_10_17_16
CTX_LH_PERICALCARINE_UCBERKELEYAV45_10_17_16
CTX_LH_POSTCENTRAL_UCBERKELEYAV45_10_17_16
CTX_LH_POSTERIORCINGULATE_UCBERKELEYAV